In [14]:
import os 
from dotenv import load_dotenv
load_dotenv()

from src.db_llm import LLMDB
from llama_index.core.vector_stores import VectorStoreQuery

from llama_index.core.schema import NodeWithScore
from typing import Optional
from llama_index.core.response.notebook_utils import display_source_node

from openai import AzureOpenAI
from llama_index.core import PromptTemplate

In [2]:
llmdb = LLMDB()

vector_store = llmdb.get_vector_store()
embed_model = llmdb.get_llm_embedding()

In [32]:
query_str = "data visualization in Python"
query_embedding = embed_model.get_query_embedding(query_str) 

query_mode = "default"
vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, 
    similarity_top_k=5, 
    mode=query_mode, 
)

In [33]:
query_result = vector_store.query(vector_store_query)

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))

context_str = " ".join([i.text for i in nodes_with_scores])

In [40]:
for node in nodes_with_scores: 
    print(type(node))

<class 'llama_index.core.schema.NodeWithScore'>
<class 'llama_index.core.schema.NodeWithScore'>
<class 'llama_index.core.schema.NodeWithScore'>
<class 'llama_index.core.schema.NodeWithScore'>
<class 'llama_index.core.schema.NodeWithScore'>


In [35]:
client = AzureOpenAI(
        api_key="25417b9e73574c49965cad8f28ab4dd6",  
        api_version="2024-02-01",
        azure_endpoint="https://openaitcuc.openai.azure.com/"
    ) 
deployment_name = 'corpu-text-gpt-4o'

template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please give the summary"
)
qa_template = PromptTemplate(template)

# you can create text prompt (for completion API)
prompt = qa_template.format(context_str=context_str)

In [36]:
llm =  AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
) 

In [37]:
message = [
    {"role":"system", "content":""},
    {"role":"user", "content":prompt}
]

In [38]:
response = llm.chat.completions.create(
    model='corpu-text-gpt-4o', 
    messages=message, 
    temperature= 0.4, 
    max_tokens=1000
)

In [39]:
response = response.choices[0].message.content.replace(' .', '.').strip()
print(response)

The provided context is an excerpt from a guide on using Python for data science, specifically focusing on data visualization with Pandas and Matplotlib. Here are the key points summarized:

1. **Plotting with Matplotlib**: 
   - You can customize bar plots using parameters like `alpha` for transparency and `title` for naming the plot.
   - DataFrames can be plotted directly, treating each column as a separate series. Horizontal bar graphs can be created using `kind='barh'` and stacked bars using `stacked=True`.

2. **Color Customization**:
   - The default number of colors in a plot is 5. To display more than 5 series, additional colors must be specified using the `color` keyword.

3. **Plot Types in Pandas**:
   - Pandas offers various plot types such as line plots, vertical and horizontal bar plots, histograms, boxplots, KDE plots, area plots, and pie plots.

4. **KDE Plot**:
   - KDE (Kernel Density Estimation) plots are used to visualize data distribution, similar to histograms bu